In [33]:
import pandas as pd
import scipy
import glob
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy
import os
from sklearn.metrics import r2_score
from scipy.stats import binom, poisson, nbinom, expon, gamma
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
import warnings
import statsmodels.api as sm
from sklearn import linear_model
from catboost import CatBoostRegressor
import itertools
import sys

sys.path.insert(0,'C:/MyDevelopment/Goalscorers')
from helper_functions import *
import data_cleaning as dc
import feature_engineering as fe

In [34]:
goal_exps = pd.read_csv("C:/MyDevelopment/Goalscorers/goal_expectancies/fbref_matched_expectancies.csv")

In [35]:
def get_pre_processed_data(seasons_to_load, leagues_to_load, data_exp):
    data = dc.load_data(seasons_to_load, leagues_to_load)
    data = dc.add_datetime(data)
    data = dc.add_opposite_team(data)
    data = dc.map_position(data)
    data = dc.add_npg(data)
    data = dc.add_year_week(data)
    data = dc.add_goal_expectancies(data,data_exp)
    data = dc.add_supremacy(data)
    data = dc.add_npxg_per_minute(data)
    data = dc.add_team_scored_and_conceded_npxg(data)
    data = dc.add_solo_striker_position(data)
    data = dc.add_main_opposing_gk(data)
    data = dc.remove_gk(data)
    data = dc.drop_NAs(data)
    
    return data

preprocessed_data = get_pre_processed_data(None, None, goal_exps)

In [36]:
##filter data
data = preprocessed_data.copy(deep=True)
data = data[((data.season == '2017-2018') | (data.season == '2018-2019') | (data.season == '2019-2020') | (data.season == '2020-2021'))]

In [37]:
data[["league_name","season"]].value_counts().sort_index()

league_name     season   
Bundesliga      2017-2018     7892
                2018-2019     7875
                2019-2020     8132
                2020-2021     8652
Championship    2018-2019    14088
                2019-2020    14356
                2020-2021    14837
Eredivisie      2018-2019     7774
                2019-2020     5913
                2020-2021     8417
La Liga         2017-2018     9785
                2018-2019     9829
                2019-2020    10181
                2020-2021    10841
Ligue 1         2017-2018     9749
                2018-2019     9772
                2019-2020     7173
                2020-2021    10725
Premier League  2017-2018     9682
                2018-2019     8747
                2019-2020     8873
                2020-2021     8663
Primeira Liga   2018-2019     7870
                2019-2020     8097
                2020-2021     8718
Serie A         2017-2018     9817
                2018-2019     9783
                2019-2020    

In [38]:
##feature engineering

#add average npxg per minute features 
data = fe.add_player_avg_feature(data, "npxg_per_min", 5)
data = fe.add_player_avg_feature(data, "npxg_per_min", 10)

#add average npxg per minute / average team conceded npxg features, for 2,6,10 previous appearances
data, team_data = fe.add_team_avg_feature(data, "team_conceded_npxg", 5)
data["npxg_per_min_over_squad_opp_avg"] = data.npxg_per_min - data.avg_team_conceded_npxg_l5 
data = fe.add_player_avg_feature(data, "npxg_per_min_over_squad_opp_avg", 5)
data = fe.add_player_avg_feature(data, "npxg_per_min_over_squad_opp_avg", 10)

#add average touches in att_3rd and touches in att_pen_area in last 5 games
data["touches_att_3rd_per_min"] = data.touches_att_3rd/data.minutes
data["touches_att_pen_area_per_min"] = data.touches_att_pen_area/data.minutes

data = fe.add_player_avg_feature(data, "touches_att_3rd_per_min", 5)
data = fe.add_player_avg_feature(data, "touches_att_pen_area_per_min", 5)
data.npxg = data.npxg.transform(lambda x: 0.0001 if x == 0 else x)

#add features
data["frac_90"] = data.minutes/90
data["ln_frac_90"] = np.log(data.frac_90)
data["ln_frac_90_start"] = data.ln_frac_90 * data.start
data["ln_frac_90_not_start"] = data.ln_frac_90 * (1.0 - data.start)
data["goal_exp_2"] = data.goal_exp ** 2
data["supremacy_2"] = data.supremacy ** 2
data["is_home"] = np.where(data.squad == data.home_team, 1, 0)
data["start"] = np.where(data.start == True, 1, 0)

In [39]:
#get_week_difference
def get_week_difference(data, current_year, current_week):
    def func(row):
        d = (current_year - row["year"]) * 52 + (current_week - row["week"])
        assert(d >= 0)
        return d
        
    diff = data.apply(func, axis = 1)
    return diff

#get training weights
def get_weights(data, year, week, decay_factor):
    if decay_factor != None:
        week_diff = get_week_difference(data, year, week)
        weights = np.exp(-decay_factor*week_diff)
    else:
        weights = np.full(len(data), 1.0)
        
    return weights

#standardize
def standardize(X, cols_to_standardize, scaler=None):
    
    if scaler==None:
        scaler = StandardScaler()
        scaler.fit(X[cols_to_standardize])
    
    X[cols_to_standardize] = scaler.transform(X[cols_to_standardize])
    return X, scaler

#estimate gamma parameters
def get_gamma_parameters(y_train, train_preds, test_preds, n_features):
            
    def estimate_x2_scale(y, mu, n_sample, dof):
        resid = np.power(y- mu, 2)
        variance = mu ** 2
        df_residuals = n_sample - dof
        return np.sum(resid / variance) / df_residuals

    inv_shape_param =  estimate_x2_scale(y_train, train_preds, len(y_train), n_features)
    shape_from_model = 1/inv_shape_param

    scales = [m_i /shape_from_model for m_i in test_preds]
    shapes = np.full(len(scales), shape_from_model)

    return scales, shapes

#get log likelihoods given expectancies
def calculate_log_likelihood(y_true, parameters, distribution='Poisson',individual_scores=False):
    assert(len(y_true) == len(parameters[0]))
    if distribution =='Poisson' or distribution == 'Exponential':
        assert(len(parameters) == 1)
    elif distribution =='Gamma':
        assert(len(parameters) == 2)
    
    if distribution=='Poisson':
        expectancies = parameters[0]
        probs = poisson.pmf(y_true, expectancies)
        ind_log_ll = np.log(probs)
        
    elif distribution=='Exponential':
        expectancies = parameters[0]
        probs = expon.pdf(y_true, scale= 1/expectancies)
        ind_log_ll = np.maximum(-10000,np.log(probs))
        
    elif distribution=='Gamma':
        scales = parameters[0]
        shapes = parameters[1]
        probs = gamma.pdf(y_true, a = shapes,scale=scales)
        ind_log_ll = np.maximum(-10000,np.log(probs))
        
    else:
        raise ValueError('Invalid distribution argument passed.')
        
    
    log_ll = np.sum(ind_log_ll)
    avg_log_ll = log_ll/len(probs)
    
    if individual_scores == False:
        return log_ll, avg_log_ll
    else:
        return ind_log_ll

In [48]:
def get_consecutive_evaluations(data, decay_factor, model_parameters, features,target_variable,
                                cols_to_standardize, cols_to_ohe, distribution, verbose=False):

    grouped = data.sort_values(["year", "week"]).groupby(["year", "week"], as_index = False)
    df_test_preds = []
    week_counter = 0
    
    for i,_ in grouped:
        if week_counter > 0:
            year = i[0]
            week = i[1]
            if verbose == True:
                print(f"Year = {year}, week = {week}...")

            #split validation data by week-year
            train_data, test_data = split_dataset(data, year, week)
            train_weights = get_weights(train_data, year, week, decay_factor)

            #get x and y data
            X_train, X_test = train_data[features].copy(deep=True), test_data[features].copy(deep=True)
            y_train, y_test = train_data[target_variable], test_data[target_variable]
            
            #standardize
            X_train, scaler = standardize(X_train, cols_to_standardize=cols_to_standardize)
            X_test, _ = standardize(X_test, cols_to_standardize=cols_to_standardize, scaler=scaler)

            #add constant
            X_train = sm.add_constant(X_train)
            X_test = sm.add_constant(X_test)

            #train model
            if distribution=="Gamma":
                model = linear_model.GammaRegressor(fit_intercept=False, alpha = model_parameters["regularization_parameter"], max_iter = 500)
                model.fit(X_train, y_train, sample_weight = train_weights)
                
            elif distribution == "Poisson":
                model = linear_model.PoissonRegressor(fit_intercept=False, alpha = model_parameters["regularization_parameter"], max_iter = 500)
                model.fit(X_train, y_train, sample_weight = train_weights)
                
            elif distribution == "Catboost":
                model = CatBoostRegressor(task_type="GPU",
                                            devices='0:1',
                                            early_stopping_rounds = 50,
                                            loss_function="Poisson",
                                            **model_parameters)
                model.fit(X_train,y=y_train, cat_features = cols_to_ohe,verbose=False, sample_weight = train_weights)
                
            else:
                raise Exception("Invalid distribution arg.")
            
            #get predictions
            train_preds = model.predict(X_train)
            test_preds = model.predict(X_test)

            #if gamma distribution find shape and scale parameters
            if distribution == "Gamma":
                scales, shapes = get_gamma_parameters(y_train, train_preds, test_preds, X_train.shape[1])
                test_data = test_data.assign(pred_exp=test_preds,
                                             pred_scale=scales,
                                             pred_shape=shapes)

                df_test_preds.append(test_data[["player_id", "home_team", "away_team", "datetime","pred_exp", "pred_scale", "pred_shape"]])

            else:
                test_data = test_data.assign(pred_exp=test_preds)
                df_test_preds.append(test_data[["player_id", "home_team", "away_team", "datetime","pred_exp"]])

        week_counter += 1
    
    df_test_pred = (
    pd
    .concat(df_test_preds, ignore_index=True)
    .merge(
        data[["player_id", "home_team", "away_team", "datetime", "npxg", "npg", "start", "position"]], 
        how="left", 
        on=["player_id", "home_team", "away_team", "datetime"], 
        validate="1:1"
    ))
    
    #add ll
    df_test_pred = df_test_pred.assign(poisson_ll = calculate_log_likelihood(df_test_pred.npg,
                                          [df_test_pred.pred_exp],
                                          distribution='Poisson',
                                          individual_scores=True),
                                       
                                       exp_ll = calculate_log_likelihood(df_test_pred.npxg,
                                          [df_test_pred.pred_exp],
                                          distribution='Exponential',
                                          individual_scores=True) 
                                      )
    
    if distribution == "Gamma":
        df_test_pred = df_test_pred.assign(gamma_ll = calculate_log_likelihood(df_test_pred.npxg,
                                          [df_test_pred.pred_scale, df_test_pred.pred_shape],
                                          distribution='Gamma',
                                          individual_scores=True))
        
    return df_test_pred, model, X_train

In [49]:
def ohe(data, categorical_columns) -> pd.DataFrame:
    data_ohe = data.copy(deep=True)
    return (
        data_ohe
        .join(pd.get_dummies(data[categorical_columns].astype(str), dtype=int))
    )

def get_features(data, cols_to_ohe):
    features = [
        'goal_exp', 'supremacy', 'frac_90'
    ]
    
    for col in cols_to_ohe:
        new_features = [f"{col}_{val}" for val in data[col].unique()]
        features = features + new_features
    
    return features

target_variable = 'npxg'
target_variable_cat = 'npg'

cols_to_standardize = [
    'goal_exp',
    'supremacy',
    'ln_frac_90_start',
    'ln_frac_90_not_start',
    'avg_npxg_per_min_l5',
    'avg_npxg_per_min_l10',
    'avg_team_conceded_npxg_l5',
    'avg_npxg_per_min_over_squad_opp_avg_l5',
    'avg_npxg_per_min_over_squad_opp_avg_l10',
    'avg_touches_att_3rd_per_min_l5',
    'avg_touches_att_pen_area_per_min_l5',
]

cols_to_ohe = [
    'position',
    'squad_opp',
    'player',
    'gk_opp'
]

cols_other = [
    'start',
    'is_home'
]

#data_model = ohe(data, cols_to_ohe)
data_catboost = data.copy()

#model_features = get_features(data_model, cols_to_ohe)
catboost_features = cols_other + cols_to_standardize + cols_to_ohe

In [50]:
TUNE = True

In [ ]:
regularization_grid = [0.001,0.01,0.1,1.0]
decay_grid = [0.001,0.01,0.1]

if TUNE==True:
    grid_scores = []
    for reg in regularization_grid:
        for dec in decay_grid:
            print(f"Trying reg={reg} and decay factor = {dec}")
            model_parameters = {'regularization_parameter':reg}
            df_test_pred, model, _ = get_consecutive_evaluations(data_model, dec, model_parameters,model_features,
                                                                 target_variable, cols_to_standardize,
                                                                 cols_to_ohe, distribution="Gamma")

            ll_exponential = df_test_pred.exp_ll.mean()
            ll_poisson =  df_test_pred.poisson_ll.mean()
            ll_gamma = df_test_pred.gamma_ll.mean()
            r2score  = r2_score(df_test_pred.npg,df_test_pred.pred_exp)

            grid_scores.append((reg, dec, r2score, ll_exponential,ll_poisson, ll_gamma))

    grid_scores = pd.DataFrame(grid_scores, columns=["reg_parameter","decay_parameter","r2score",
                                                         "ll_exponential","ll_poisson", "ll_gamma"])
    
    grid_scores.to_pickle("df_tuning_results.pickle")
    
else:
    grid_scores = pd.read_pickle("df_tuning_results.pickle")

In [54]:
model_parameters

{'iterations': 50,
 'depth': 6,
 'border_count': 50,
 'learning_rate': 0.001,
 'l2_leaf_reg': 0.001}

In [58]:
pd.DataFrame(hyperparameter_combs)

,iterations,depth,border_count,learning_rate,l2_leaf_reg
0,50,6,50,0.001,0.001


In [61]:
hyperparameter_grid = {
    'iterations': [50, 100, 200,400],
    'depth': [6,10,12,16],
    'border_count': [50],#,100,200],
    'learning_rate': [0.1],
    'l2_leaf_reg': [0.01]
}
decay_grid = [0.001,0.01,0.1]
train_data_catboost = data_catboost[(data.season == '2017-2018')]

def make_grid(param_grid):  
    keys=param_grid.keys()
    combinations=itertools.product(*param_grid.values())
    ds=[dict(zip(keys,cc)) for cc in combinations]
    return ds

if TUNE==True:
    hyperparameter_combs = make_grid(hyperparameter_grid)
    grid_params = []
    grid_scores = []
    
    for model_parameters in hyperparameter_combs:
        for dec in decay_grid:
            print(f"Trying params={model_parameters} and decay factor = {dec}")
            
            df_test_pred, model, _ = get_consecutive_evaluations(train_data_catboost, dec, model_parameters,
                                                                 catboost_features,target_variable, 
                                                                 cols_to_standardize,cols_to_ohe,
                                                                 distribution="Catboost",
                                                                 verbose=True)

            ll_poisson =  df_test_pred.poisson_ll.mean()
            r2score  = r2_score(y_true=df_test_pred.npg,y_pred=df_test_pred.pred_exp)

            grid_scores.append((dec, r2score, ll_exponential,ll_poisson))
            grid_params.append(model_parameters)

    grid_scores = pd.DataFrame(grid_scores, columns=["decay_parameter","r2score","ll_exponential","ll_poisson"])
    grid_params = pd.DataFrame(grid_params)
    
    grid_scores = pd.concat([grid_params, grid_scores],axis=1).sort_values()
    grid_scores.to_pickle("df_tuning_results.pickle")
    
else:
    grid_scores = pd.read_pickle("df_tuning_results.pickle")

Trying params={'iterations': 50, 'depth': 6, 'border_count': 50, 'learning_rate': 0.1, 'l2_leaf_reg': 0.01} and decay factor = 0.001
Year = 2017, week = 32...
Year = 2017, week = 33...
Year = 2017, week = 34...
Year = 2017, week = 36...
Year = 2017, week = 37...
Year = 2017, week = 38...
Year = 2017, week = 39...
Year = 2017, week = 41...
Year = 2017, week = 42...
Year = 2017, week = 43...
Year = 2017, week = 44...
Year = 2017, week = 46...
Year = 2017, week = 47...
Year = 2017, week = 48...
Year = 2017, week = 49...
Year = 2017, week = 50...
Year = 2017, week = 51...
Year = 2017, week = 52...
Year = 2018, week = 1...
Year = 2018, week = 2...
Year = 2018, week = 3...
Year = 2018, week = 4...
Year = 2018, week = 5...
Year = 2018, week = 6...
Year = 2018, week = 7...
Year = 2018, week = 8...
Year = 2018, week = 9...
Year = 2018, week = 10...
Year = 2018, week = 11...
Year = 2018, week = 13...
Year = 2018, week = 14...
Year = 2018, week = 15...
Year = 2018, week = 16...
Year = 2018, week 

In [62]:
grid_scores

,iterations,depth,border_count,learning_rate,l2_leaf_reg,decay_parameter,r2score,ll_exponential,ll_poisson
0,50,6,50,0.1,0.01,0.001,0.118115,-0.134466,-0.276929
1,50,6,50,0.1,0.01,0.010,0.118605,-0.134466,-0.276825
2,50,6,50,0.1,0.01,0.100,0.117408,-0.134466,-0.277203


In [ ]:
decay_param = 0.01
reg_param = 0.001

df_test_pred, model, _ = get_consecutive_evaluations(data_model, decay_param, reg_param,model_features,target_variable,
                                                        cols_to_standardize,distribution="Gamma")

In [ ]:
dict(zip(["const"] + model_features, model.coef_))

In [ ]:
model_results = expectancies.reset_index(drop=True)

In [ ]:
model_results_train = pd.merge(model_results,
                         train_data[["datetime","date","squad","position_mapped", "start", "player","npxg","npg"]],
                         on = ["date","squad","player"],
                         how='inner')


model_results_validation = pd.merge(model_results,
                         val_data[["datetime","date","squad","position_mapped", "start", "player","npxg","npg"]],
                         on = ["date","squad","player"],
                         how='inner')

In [ ]:
model_results_validation.head(5)

In [ ]:
#get exponential fit ll
ind_llres = get_log_likelihood(model_results_validation.npxg,
                               (model_results_validation.scale, model_results_validation["shape"]), 
                               distribution='Gamma',
                               individual_scores=True)

llres = get_log_likelihood(model_results_validation.npxg,
                           (model_results_validation.scale, model_results_validation["shape"]), 
                           distribution='Gamma',
                           individual_scores=False)

r2score = r2_score(model_results_validation.npxg, model_results_validation.expectancies)
print(llres, r2score)
model_results_validation["ll"] = ind_llres

#get poisson fit ll 
ind_llres = get_log_likelihood(model_results_validation.npg,
                               (model_results_validation.expectancies,), 
                               distribution='Poisson',
                               individual_scores=True)

llres = get_log_likelihood(model_results_validation.npg,
                           (model_results_validation.expectancies,), 
                           distribution='Poisson',
                           individual_scores=False)

print(llres)
model_results_validation["ll_poisson"] = ind_llres

In [ ]:
def feature_mapping_function(x_train,x_val, encoding_dict):    
    
    #one hot encode x_train and x_test
    x_train, encoder = one_hot_encode_data(x_train, encoding_dict=encoding_dict)
    x_val, _ = one_hot_encode_data(x_val, encoding_dict=encoding_dict, preprocessor=encoder)
    
    #multiply npxg by OHE variables and drop npxg col
    x_train = x_train.multiply(x_train["expectancies_log"], axis="index")
    x_train.expectancies_log = np.sqrt(x_train.expectancies_log)
    
    x_val = x_val.multiply(x_val["expectancies_log"], axis="index")
    x_val.expectancies_log = np.sqrt(x_val.expectancies_log)
    
    return x_train, x_val    

In [ ]:
def compute_avg_pmf(df, sample_size, pmf_support):   
    # Initialize array to store average PMFs
    avg_pmfs = np.zeros((len(df), 10))
    
    # Generate gamma samples in bulk for all rows
    gamma_samples = gamma.rvs(a=df['shape'].values[:, np.newaxis], scale=df['scale'].values[:, np.newaxis], size=(len(df), sample_size))
    
    # Compute Poisson PMFs for all gamma samples (vectorized operation)
    sample_pmfs = poisson.pmf(pmf_support[np.newaxis, np.newaxis, :], gamma_samples[:, :, np.newaxis])
    
    # Compute the average PMF across all samples (axis=1)
    avg_pmfs = np.mean(sample_pmfs, axis=1)
    
    return avg_pmfs

In [ ]:
#compare PMF using single point estimate for xg (expectancies)
x = np.arange(0, 6)
probs= poisson.pmf(x, model_results_validation.expectancies.values[:,np.newaxis])
probs_mean = np.mean(probs, axis=0)
true_dist = model_results_validation.npg.value_counts(normalize=True).sort_index()

plt.plot(x,probs_mean, label="Pred ngp")
true_dist.plot(label="True npg")
plt.legend()

[print(i) for i in enumerate(list(zip(probs_mean, true_dist.values)))]

plt.show()

#compare PMF using sampled estimate
sampled_distributions = compute_avg_pmf(model_results_validation, sample_size=4000, pmf_support=np.arange(0,20))

In [ ]:
ben_comparison = model_results_validation[(model_results_validation.datetime >= '2020-09-19 16:30:00') &\
                                          (model_results_validation.start == True)]

In [ ]:
ben_comparison["ll_poisson"].sum()

In [ ]:
position = 'FW'
df_position = ben_comparison[ben_comparison.position_mapped == position]
df_position.ll_poisson.sum()

In [ ]:
model_results_train.to_csv("C:/MyDevelopment/Goalscorers/model_results/npxg_train_predictions.csv", index=False)
model_results_validation.to_csv("C:/MyDevelopment/Goalscorers/model_results/npxg_test_predictions.csv", index=False)